# Humid Heat Metrics

Humid heat metrics index temperature and humidity, which is often more useful than comparing either variable alone when considering perceived temperature as they together affect the human body's ability to cool itself down. Humid heat metrics are especially important for the safety of outside workers, the elderly, or otherwise high-risk high-exposure individuals. 

Wet Bulb Globe Temperature (WBGT) is a measure of heat stress. The equation for outdoor ($WBGT_{OD}$) and indoor/shaded ($WBGT_{ID}$) WBGTs are:

$WBGT_{od} = 0.7*T_{nwb} + 0.2*T_g + 0.1*T_a$

$WBGT_{id} = 0.7*T_{nwb} + 0.3*T_g$

where $T_a$ refers to Dry Bulb Ambient Temperatue, $T_{nwb}$ is the Natural Wet Bulb Temperature wet bulb temperature with exposure to wind and sun, and $T_g$ is the Globe Temperature temperature taken from inside a copper globe painted black and exposed to the sun.

However this formula is complicated by the reality that Natural Wet Bulb Temperature and Globe Temperature are not always readily available variables from weather stations or atmospheric models.

In this notebook we will demonstrate the Australian Bureau of Meteorology (ABM) and Bernard methods of predicting wet bulb global temperature with a focus on the July 1995 Chicago heatwave.

For our analysis, we have ERA5 renanalysis data for the lower contiguous United States (50N, 24S, -66E, -125W) from July 1995 with the variables: 2-meter temperature, 2-meter dew point temperature, surface pressure, and *u*/*v* wind components.

In [ ]:
import geocat.datafiles as gcd
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [ ]:
era5_file = "../../adaptor.mars.internal-1721674223.0932782-17053-4-ca11ba26-b36b-4a33-902e-b4ed0d92364e.nc"
era5 = xr.open_dataset(era5_file)
era5

## Australian Bureau of  (ABM)

Method of estimating Wet Bulb Global Temperature

- tends to overpredict WBGT compared to other models
- assumes full sunlight and light breeze

In [ ]:
def calc_abm_wbgt(t_a, rh):
    e = (rh / 100)**17.27 * (t_a/(337.7 * t_a)) # water vapor pressure [hPa]
    wbgt = (0.567 * t_a) + (0.393 * e) + 3.94
    return wbgt

To use our ERA5 data in this equation, we need to first approximate relative humidity using the Clausius-Clapeyron Equation:

In [ ]:
def _calc_saturation_vapor_pressure(T_a): # Magnus-Tetens Approximation
    e_sat = 6.11 * np.exp((17.625 * T_a) / (T_a + 243.04)) # Saturation Vapor Pressure in hPa
    return e_sat

def calc_relative_humidity_era5(T_a, T_d):
    e_sat = _calc_saturation_vapor_pressure(T_a) # saturation vapor pressure
    e = 6.11*np.exp(17.625 * T_d / (243.04 + T_d)) # vapor pressure from dew point temp
    rh = 100 * e / e_sat  # Clausius-Clapeyron equation
    return rh

rh = calc_relative_humidity_era5(era5.t2m, era5.d2m)

In [ ]:
wbgt_abm = calc_abm_wbgt(era5.t2m, rh)
wbgt_abm

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.PlateCarree())

c = plt.contourf(era5.longitude, era5.latitude, wbgt_abm.sel(time='1995-07-14T12'))

ax.coastlines()
ax.gridlines(draw_labels=True)

cbar = plt.colorbar(c, ax=ax, orientation='horizontal', pad=0.1, shrink=0.8)
cbar.set_label('Wet Bulb Globe Temperature (WBGT)')

ax.set_title('July 14, 1995 noon - ABM WBGT method')

## Bernard

Bernard is a nother method of calculating WBGT that incorporates wind speed.

Thermodynamic Wet Bulb Temperature ($T_{pwb}$) wet bulb temperature in the shade and fanned or rotated. This is the wet bulb typically used for dew point calculations.


In [ ]:
def calc_bernard_wbgt(t_a, t_d, v):
    
    if np.all(v < 0.3):
        return np.nan  # Return NaN where velocity is below the threshold
    elif np.all((0.3 <= v) & (v <= 3)):
        wbgt = (0.67 * t_d) + (0.33 * t_a) - (0.48 * np.log10(v) * (t_a - t_d))
    else:
        wbgt = (0.7 * t_d) + (0.3 * t_a)
    
    return wbgt


First we will need to combine our u and v windspeeds:

In [ ]:
v = np.sqrt(era5.u10**2 + era5.v10**2)

In [ ]:
wbgt_bernard = calc_bernard_wbgt(era5.t2m, era5.d2m, v)
wbgt_bernard

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.PlateCarree())

c = plt.contourf(era5.longitude, era5.latitude, wbgt_bernard.sel(time='1995-07-14T12'))

ax.coastlines()
ax.gridlines(draw_labels=True)

cbar = plt.colorbar(c, ax=ax, orientation='horizontal', pad=0.1, shrink=0.8)
cbar.set_label('Wet Bulb Globe Temperature (WBGT)')

ax.set_title('July 14, 1995 noon - Bernard WBGT method')

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.PlateCarree())

diff = wbgt_abm.sel(time='1995-07-14T12') - wbgt_bernard.sel(time='1995-07-14T12')
c = plt.contourf(era5.longitude, era5.latitude, diff)

ax.coastlines()
ax.gridlines(draw_labels=True)

cbar = plt.colorbar(c, ax=ax, orientation='horizontal', pad=0.1, shrink=0.8)
cbar.set_label('Wet Bulb Globe Temperature (WBGT)')

ax.set_title('July 14, 1995 noon - WBGT Method Difference(ABM - Bernard)')